In [ ]:
import os
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Compile landmarks Dataset

In [ ]:
words = [
    'آئو', 'آئوں', 'آئی', 'آئیں',
    'آئے', 'آپ', 'آیا', 'اپنا',
    'تم', 'تھا', 'جائیں', 'میں',
    'نام', 'وہ', 'کا', 'کام',
    'کرو', 'کیا', 'گیا', 'گے',
    'ہم', 'ہے',
]

In [ ]:
word2paths = {
    word : glob(
            os.path.join(
                os.path.abspath(os.sep),
                'content',
                'drive',
                'Othercomputers',
                'My Laptop',
                'Pakistani Sign Language Translation with 3D Pose Estimation and Attention Models',
                'Datasets',
                'Video_Data',
                'PSL_Landmarks',
                '*',
                '*',
                '*',
                f'{word}_person*.csv'
            )
        )
    for word in words
}

word2num_files = {w:len(ps) for w, ps in word2paths.items()}
word2num_files

{'آئو': 0,
 'آئوں': 0,
 'آئی': 0,
 'آئیں': 0,
 'آئے': 0,
 'آپ': 0,
 'آیا': 0,
 'اپنا': 192,
 'تم': 176,
 'تھا': 176,
 'جائیں': 0,
 'میں': 0,
 'نام': 192,
 'وہ': 192,
 'کا': 176,
 'کام': 176,
 'کرو': 176,
 'کیا': 176,
 'گیا': 176,
 'گے': 176,
 'ہم': 176,
 'ہے': 176}

In [ ]:
word2paths[words[-1]][0].split(os.sep)

['',
 'content',
 'drive',
 'Othercomputers',
 'My Laptop',
 'Pakistani Sign Language Translation with 3D Pose Estimation and Attention Models',
 'Datasets',
 'Video_Data',
 'PSL_Landmarks',
 'Pose_World_Landmarks',
 'HFAD_Book1',
 'Person254',
 'ہے_person254_below.csv']

In [ ]:
header = ['Word', 'Person_Number', 'Camera_Angle', 'Pose_World_Landmarks', 'Left_Hand_World_Landmarks', 'Right_Hand_World_Landmarks','Pose_Landmarks','Left_Hand_Landmarks','Right_Hand_Landmarks']
N_HAND_COLS = 21*3

# group paths by landmark_type
filename2paths = {}
for word, paths in word2paths.items():
    for path in paths:
        filename = os.path.split(path)[1].split('.')[0]
        try:
            filename2paths[filename].append(path)
        except:
            filename2paths[filename] = [path]

target_filepath = os.path.join(
    os.path.abspath(os.sep),
    'content',
    'drive',
    'Othercomputers',
    'My MacBook Pro',
    'SignLanguageTranslator',
    'SignLanguageTranslator',
    'datasets',
    'Signs_recordings',
    'mini_mediapipe_landmarks_dataset.csv'
)

# append row
with open(target_filepath, 'w') as fout:
    fout.write(','.join(header)+'\n')

    for filename, paths in tqdm(filename2paths.items()):

        word, person, camera, pose_3d, lhand_3d, rhand_3d, pose_2d, lhand_2d, rhand_2d = None, None, None, None, None, None, None, None, None

        word, person, camera = filename.split('.')[0].split('_')
        person = person.lstrip('person')

        for filepath in paths:
            landmarks = np.loadtxt(filepath, delimiter=',', skiprows=1)

            landmark_type = filepath.split(os.sep)[-4]

            if landmark_type == 'Pose_World_Landmarks':
                pose_3d = landmarks.tolist()

            elif landmark_type == 'Hand_World_Landmarks':
                # landmarks[:, 0] *= -1

                lhand_3d = landmarks[:, :N_HAND_COLS].tolist()
                rhand_3d = landmarks[:, N_HAND_COLS:].tolist()

            elif landmark_type == 'Pose_Landmarks':
                pose_2d = landmarks.tolist()

            elif landmark_type == 'Hand_Landmarks':
                # landmarks[:, 0] *= -1

                lhand_2d = landmarks[:, :N_HAND_COLS].tolist()
                rhand_2d = landmarks[:, N_HAND_COLS:].tolist()

            else:
                print(filename, 'has weird landmark_type:', landmark_type)

        if None in [word, person, camera, pose_3d, lhand_3d, rhand_3d, pose_2d, lhand_2d, rhand_2d]:
            print(filename, ' is missing a landmark_type')
        else:
            fout.write(f'{word},{person},{camera},"{pose_3d}","{lhand_3d}","{rhand_3d}","{pose_2d}","{lhand_2d}","{rhand_2d}"\n')

# compress in the worst way possible
df = pd.read_csv(target_filepath).sort_values(by=header[:3])
df.to_csv(target_filepath, compression='zip', index=False)

target_zipfilepath = target_filepath[:-4]+'.zip'
os.rename(target_filepath, target_zipfilepath)

df.head(2)

100%|██████████| 584/584 [00:57<00:00, 10.12it/s]


,Word,Person_Number,Camera_Angle,Pose_World_Landmarks,Left_Hand_World_Landmarks,Right_Hand_World_Landmarks,Pose_Landmarks,Left_Hand_Landmarks,Right_Hand_Landmarks
0,اپنا,person101,below,"[[-0.023365654051303864, -0.5074993371963501, ...","[[-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.5712956190109253, 0.14683382213115692, -0....","[[-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,اپنا,person101,front,"[[0.02768453024327755, -0.5309510231018066, -0...","[[0.015320802107453346, -0.02556394599378109, ...","[[0.005956127308309078, -0.024160169064998627,...","[[0.49605792760849, 0.1614334136247635, -0.948...","[[-0.3604905307292938, 0.4612244665622711, 1.8...","[[0.6788578629493713, 0.45441576838493347, 1.9..."


In [ ]:
!ls -l {'"'+os.path.split(target_filepath)[0]+'"'}

total 167198
-rw------- 1 root root 171210439 Dec  3 17:20 mini_mediapipe_landmarks_dataset.zip


# list down recording_labels

In [ ]:
ref_clips_paths = glob(
    os.path.join(
        os.path.abspath(os.sep),
        'content',
        'drive',
        'Othercomputers',
        'My Laptop',
        'Pakistani Sign Language Translation with 3D Pose Estimation and Attention Models',
        'Miscellaneous',
        'Video_Data_Collection',
        'ReferenceClips',
        'HFAD_Book1',
        '*.MP4'
    )
)
ref_clips_filenames = [os.path.split(p)[1] for p in ref_clips_paths]
ref_clips_names = [p.split('.')[0] for p in ref_clips_filenames]
assert all([len(p.split('.'))==2 for p in ref_clips_filenames]), 'some word contains a . in it'
ref_clips_names.remove('مالٹاOrange old')

In [19]:
target_filepath = os.path.join(
    os.path.abspath(os.sep),
    'content',
    'drive',
    'Othercomputers',
    'My MacBook Pro',
    'SignLanguageTranslator',
    'SignLanguageTranslator',
    'datasets',
    'signs_recordings',
    '*'
)
glob(target_filepath)

In [23]:
data = {
    'None':['None'],
    'HFAD_Book1':ref_clips_names
}
with open(target_filepath, 'w') as f:
    f.write(json.dumps(data, indent=4, sortkeys=True))

FileNotFoundError: ignored

In [ ]:
# glob entire dataset (csv)
# save paths into file
# shuffle
# tqdm & append n_frames to list
# hist